# Launch Sites Locations Analysis

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

Previously, we did some exploratory data analysis by visualizing the SpaceX launch dataset using matplotlib and seaborn and we discovered some preliminary correlations between the launch site and success rates. Here, we will be displaying interactive geospacial visual analytics using Folium.

## Objectives
- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

Through these visualizations, we should be able to find some geographical patterns about launch sites.

### Importing Dependencies

In [2]:
import pandas as pd

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import wget

### Loading Dataset

> The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [3]:
spacex_df = pd.read_csv('Data/spacex_launch_geo.csv')
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356


### Marking all launch sites on a map
First, let's try to add each site's location on a map using site's latitude and longitude coordinates

In [4]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launchsites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launchsites_df = launchsites_df[['Launch Site', 'Lat', 'Long']]
launchsites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


The above coordinates are just plain numbers that can not give us any intuitive insights about where those launch sites are. If one is used to dealing with geographic coordinates, one could interpret those numbers directly in their mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first start by creating a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas. This first close up is for demonstrative purposes only.

In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Space Center's Co-ordinate with a popup label showing its name.
circle = folium.Circle(nasa_coordinate, radius=1000, color='#008eed', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Space Center's Co-ordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create a icon as a text label
    icon= DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color: #008eed;"><b>%s</b></div>'%'NASA JSC'
    )
)

site_map.add_child(circle)
site_map.add_child(marker)
site_map

Let's add a circle for each launch site in data frame <code>launch_sites</code>

In [8]:
# Initial the Map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# defining a function for each launch site, add a circle object based on its co-ordinate values as well as a popup label.
def add_launch_sites(row):
    circle = folium.Circle([launchsites_df['Lat'][row], launchsites_df['Long'][row]], radius=50, color='#d35400', fill=True).add_child(folium.Popup(launchsites_df['Launch Site'][row]))
    marker = folium.Marker([launchsites_df['Lat'][row], launchsites_df['Long'][row]],
    
        # create an icon as text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launchsites_df['Launch Site'][row] 
            )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

for row in range(len(launchsites_df)):
    add_launch_sites(row)

site_map

> Observation

- We can able to find that most of the launch sites are proximity towards the Equator line.
- Also find that Launch sites are very close proximity to the coastal line. 

### Marking the Sucess/Failed Launches for each site on the map

Adding the launch outcome for each site and see which sites have high success rates.

In [13]:
# creating a marker object
marker_cluster = MarkerCluster()

# for each row in dataframe, we create a marker object with its co-ordinate
for index, record in spacex_df.iterrows():
    marker = folium.Marker([record[1],record[2]], icon=folium.Icon(color='white', icon_color=record[3], icon='rocket', prefix='fa'))
    marker_cluster.add_child(marker)

# add marker cluster to current map
site_map.add_child(marker_cluster)

Let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [14]:
# create a marker object
marker_cluster = MarkerCluster()

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Finally, we will display all launches at their respective site and color coded according to their final outcome. It should be relatively simple to have an idea of the launch site's average success rate based on these markers.

In [16]:
# for each row in dataframe, creating a marker object and customize marker icon property to the outcome.
for index, record in spacex_df.iterrows():
    marker = folium.Marker([record[1],record[2]], icon=folium.Icon(color='white', icon_color=record[4],prefix='fa',icon='rocket'))
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map

### Distance between a launch site to its proximities

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as a railway or city).

In [17]:
# Add mouse position to get the co-ordinate (Lat, Long) for a mouse over on the map

formatter = "function(num){return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator='Long:',
    empty_string='NaN',
    lng_first= False,
    num_digits= 20,
    prefix='Lat:',
    lat_formatter= formatter,
    lng_formatter= formatter
)

site_map.add_child(mouse_position)
site_map

You can calculate the distance between two points on the map based on their Lat and Long values using the following method:

In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1= radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2= radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distance = R * c
    return distance

Mark down a point on the closest railway using MousePosition and calculate the distance between the railway point to the launch site.  After obtained its coordinate, create a folium.Marker to show the distance

In [19]:
railway_lat, railway_lon = 28.57295, -80.65392
launch_site_lat, launch_site_lon = launchsites_df['Lat'][2], launchsites_df['Long'][2]
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# creating marker
distance_marker = folium.Marker(
    [railway_lat,railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )

Draw a PolyLine between a launch site to the selected

In [20]:
# Create a `folium.PolyLine` object using the railway point coordinate and launch site coordinate
lines=folium.PolyLine([(railway_lat,railway_lon),(launch_site_lat,launch_site_lon)], weight=1)
site_map.add_child(lines)
site_map